In [ ]:
import base64
import operator, textwrap, dotenv

dotenv.load_dotenv()

from langchain.chat_models import init_chat_model
from langgraph.types import Send, interrupt
from typing_extensions import Annotated

from langgraph.graph import END, START, StateGraph
from typing import TypedDict
import subprocess

# 오픈 AI 직접 사용 하는 방식.
from openai import OpenAI

llm = init_chat_model("openai:gpt-4o-mini")


class State(TypedDict):
    video_file: str
    audio_file: str
    transcription: str
    summaries: Annotated[list[str], operator.add]
    thumbnail_prompt: Annotated[list[str], operator.add]
    thumbnail_sketches: Annotated[list[str], operator.add]
    mega_summarys: str
    user_feedback: str
    chosen_prompt: str


def extract_audio(state: State):
    return
    output_file = state["video_file"].replace(".mp4", ".mp3")
    command = [
        "ffmpeg",
        "-i",
        state["video_file"],
        "-filter:a",
        "atempo=2.0",
        "-y",
        output_file,
    ]
    subprocess.run(command)
    return {
        "audio_file": output_file,
    }


def transcribe_audio(state: State):
    return
    client = OpenAI()
    with open(state["audio_file"], "rb") as audio_file:
        transcription = client.audio.transcriptions.create(
            model="whisper-1",
            response_format="text",
            file=audio_file,
            language="ko",
            prompt="유발하라리, 넥서스, 책 요약"
        )
        return {
            "transcription": transcription
        }


def dispatch_summarizers(state: State):
    transcription = state["transcription"]
    chunks = []
    for i, chunk in enumerate(textwrap.wrap(transcription, 500)):
        chunks.append({
            "id": i + 1,
            "chunk": chunk
        })
    return [Send("summarize_chunk", chunk) for chunk in chunks]


def summarize_chunk(chunk):
    chunk_id = chunk["id"]
    chunk_text = chunk["chunk"]
    response = llm.invoke(
        f"""
        이 텍스트들을 보기좋게 요약을 해주세요. 한국어 입니다.
        텍스트: {chunk_text}
        """
    )
    summary = f"[Chunk {chunk_id}] {response.content}"
    return {
        "summaries": [summary]
    }


def mega_summary(state: State):
    all_summaries = "/n".join(state["summaries"])
    prompt = f"""
    요약본을 바탕으로 핵심키워드를 추출해서 요약본을 설명하는 요약본을 만들어줘 약 500자 내외면 좋겠어.
    요약본:{all_summaries}
    """
    response = llm.invoke(prompt)
    return {"mega_summarys": response.content}


def dispatch_artists(state: State):
    return [
        Send(
            "generate_thumbnail", {
                "id": i,
                "summary": state["mega_summarys"]
            }
        ) for i in [1, 2, 3]
    ]


def generate_thumbnail(args):
    concept_id = args["id"]
    summary = args["summary"]

    prompt = f"""
    이 요약본을 바탕으로, GPT images 생성기에 입력할 유튜브 썸네일을 만들기 위한 프롬프트를 만들어줘,
    - 핵심적인 시각적 요소가 2~3개 포함되어야 하고 너무 많은 요소가 들어가지 않도록 해줘,
    - 색상의 균형이 중요해,
    - 타이틀이 들어갈 위치에 핵심키워드를 넣어줘.
    요약본 : {summary}
    """

    response = llm.invoke(prompt)

    thumbnail_prompt = response.content

    client = OpenAI()

    result = client.images.generate(
        model="gpt-image-1",
        prompt=thumbnail_prompt,
        quality="low",
        moderation="low",
        size="auto"
    )

    image_bytes = base64.b64decode(result.data[0].b64_json)
    filename = f"thumbnail_{concept_id}.jpg"

    with open(filename, "wb") as file:
        file.write(image_bytes)

    return {
        "thumbnail_prompt": [thumbnail_prompt],
        "thumbnail_sketches": [filename],
    }


def human_feedback(state: State):
    answer = interrupt(
        {
            "chosen_thumbnail": "어떤 썸네일이 제일 마음에 드시나요? ",
            "feedback": "최종 썸네일을 제작하기 위한 피드백을 주세요"
        }
    )
    user_feedback = answer["user_feedback"]
    chosen_prompt = answer["chosen_prompt"]
    return {
        "user_feedback": user_feedback,
        "chosen_prompt": state["thumbnail_prompt"][chosen_prompt - 1],
    }


def generate_hd_thumbnail(state: State):
    chosen_prompt = state["chosen_prompt"]
    user_feedback = state["user_feedback"]

    prompt = f"""
    당신은 전문 유튜브 썸네일 디자이너입니다. 원본 썸네일 프롬프트를 가지고 사용자의 구체적인 피드백을 통합하여 개선된 버전을 만드세요.

    **원본 프롬프트 (ORIGINAL PROMPT):**
    {chosen_prompt}

    **통합할 사용자 피드백 (USER FEEDBACK TO INCORPORATE):**
    {user_feedback}

    다음과 같은 내용을 담아 개선된 프롬프트를 만드세요:
    1.  원본 프롬프트의 **핵심 개념을 유지**합니다.
    2.  사용자가 요청한 피드백을 **구체적으로 다루고 구현**합니다.
    3.  전문 유튜브 썸네일 제작 사양을 추가합니다:
        * **고대비** 및 **굵고 선명한 시각적 요소**
        * 시선을 사로잡는 **명확한 초점**
        * **전문적인 조명과 구도**
        * 가장자리에서 충분히 떨어지도록 **여백(패딩)을 넉넉하게 적용**한 **최적의 텍스트 배치 및 가독성**
        * **주목을 끄는 톡톡 튀는 색상**
        * **작은 썸네일 크기에서도 잘 작동하는 요소**
        * **중요:** **항상** 텍스트와 이미지 경계 사이에 **충분한 여백/패딩을 확보**하세요.
    """

    hd_response = llm.invoke(prompt)

    final_thumbnail_prompt = hd_response.content

    client = OpenAI()

    result = client.images.generate(
        model="gpt-image-1",
        prompt=final_thumbnail_prompt,
        quality="high",
        moderation="low",
        size="auto",
    )

    image_bytes = base64.b64decode(result.data[0].b64_json)

    with open("thumbnail_final.jpg", "wb") as file:
        file.write(image_bytes)



In [ ]:
from langgraph.checkpoint.memory import InMemorySaver

graph_builder = StateGraph(State)

(
    graph_builder

    .add_node("extract_audio", extract_audio)
    .add_node("transcribe_audio", transcribe_audio)
    .add_node("dispatch_summarizers", dispatch_summarizers)
    .add_node("summarize_chunk", summarize_chunk)
    .add_node("mega_summary", mega_summary)
    .add_node("dispatch_artists", dispatch_artists)
    .add_node("generate_thumbnail", generate_thumbnail)
    .add_node("human_feedback", human_feedback)
    .add_node("generate_hd_thumbnail", generate_hd_thumbnail)

    .add_edge(START, "extract_audio")
    .add_edge("extract_audio", "transcribe_audio")
    .add_conditional_edges("transcribe_audio", dispatch_summarizers, ["summarize_chunk"])
    .add_edge("summarize_chunk", "mega_summary")
    .add_conditional_edges("mega_summary", dispatch_artists, ["generate_thumbnail"])
    .add_edge("generate_thumbnail", "human_feedback")
    .add_edge("human_feedback", "generate_hd_thumbnail")
    .add_edge("generate_hd_thumbnail", END)
)

memory = InMemorySaver()
graph = graph_builder.compile(checkpointer=memory)

config = {
    "configurable": {
        "thread_id": "1",
    },
}

In [ ]:
from langgraph.types import Command

display(graph)
graph.invoke(
    {'video_file': 'video.mp4',
     'audio_file': 'video.mp3',
     'transcription': '네, 안녕하세요. 오늘은, 오늘 책도, 오늘 아침에 읽은 책도, 여기 보이시죠? 넥서스입니다, 넥서스. 오늘도 제가 아침에 책을 읽었고요. 이렇게 1차적으로는 수첩에다가 이렇게 적습니다. 1차적으로 좋은 문장들을 적고, 이렇게 문장들을 적은 다음에 적고, 제가 챕터별로 딱 3개의 문장만 뽑아서 살짝 제 스타일대로 요약을 합니다. 요약을 하고 나서 질문 하나씩 적고, 답변을 다는 방식으로 이렇게 준비를 하고 있습니다. 지금 이틀 정도 해봤는데, 상당히 좋은 것 같아요. 책의 내용도 많이 기억에 남고, 요약을 하다보니까 더 많은 생각을 할 수 있게 되고, 그런 것 같습니다. 그래서 일단 잠깐 보시면은, 짜잔, 넥서스죠? 저번 1편에서도 보여드리긴 했는데, 이번 하루의 책이고요. 오늘은 목차를 보면은, 저번 영상에서는 프로로그를 담았고, 이번에는 정보란 무엇인가, 정보가 뭔지, 그리고 이야기, 이야기에 대한 두 장을 제가 요약을 했고요. 보시면은, 정보란 무엇인가, 여기서 인상 깊었던 구절을 제가 조금 변형해서 적어봤습니다. 책 이름이 넥서스인데, 궁극적으로는 각 개인은 서로 다른 세계관을 가지며, 그것은 그 사람의 성격과 인생사가 교차하면 형성된다. 정보란 무엇인가잖아요? 정보는 사회적 연결고리, 넥서스라는 거죠. 그러니까 종이랑 종이랑 붙이는 풀처럼 사람들을 연결해주는 그런 사회적 연결고리가 넥서스고, 그것도 정보를 정의한다고 얘기를 하는 것 같습니다. 사람들은 정말 서로 다른 세계관을 가지죠. 그래서 여러분들의 세계관은, 그 세계관에서는 무엇이 가치 있다고 생각하시는지, 그걸 한번 생각해보시면 좋을 것 같아요. 네, 두 번째입니다. 이 문장도 참 좋았는데요. 과학적이나 정치 연설 같은 몇 가지 유형의 정보는, 사람들에게 현실을 정확하게 표현해서 사람들을 하나로 묶으려고 하죠. 그런데 사람들을 묶는데 효과적인 정보는 이런 사실이 아니고, 허구인 매력적인 정보가 오히려 사람들에게 잘 퍼지고, 사람들을 잘 묶는다는 얘기죠. 그래서 좋은 정보는, 정보를 사람들을 묶는 매개체로 생각해본다고 하면은, 좋은 정보는 정확한 정보가 아닐 때가 많다는 거죠. 그래서 여러분들에게 허구로 보이지만, 저거 진짜 거짓말인데, 그것도 많은 사람들이 함께 믿고 있는 정보가 있다면 뭐가 있을지 생각해보시면 좋을 것 같아요. 네, 다음입니다. 결국 역사적으로 보면, 지금 호모사피엔스가 다른 종들을 다 정복하고, 상당한 힘을 가지고 있잖아요. 그 호모사피엔스가 세계를 정복한 이유는, 거짓, 오류, 환상을 믿는 능력을 바탕으로 많은 개개인들이 연결되어 개인이 할 수 없는 거대한 일을 할 수 있었기 때문이다. 이런 문장도 중요하죠. 그래서 저도 20대 때 종교단체에 오래 있었는데, 때는 외롭지 않았어요. 신화를 바탕으로 많은 사람들이 항상 만났기 때문에 외롭지 않았는데, 점점 종교에 나오고, 진실을 추구하고, 과학을 공부하면서 외로워지는 부분이 있는 것 같아요. 그럼에도 계속 진실을 추구해야 하는 이유는 여러분들은 뭐라고 생각하시는지, 혹은 비록 거짓일지라도, 외로움보다는 거짓대로 사는 것이 나은 것인지, 이런 부분들을 좀 생각해보시면 좋을 것 같습니다. 그래서 정보란 무엇인가? 다음에서 세 가지 질문을 뽑아봤고요. 그래서 정보, 정보란 무엇일까? 이 책에서 정보라는 키워드를 많이 다루는데, 정보는 이제 사람들은 결국 나랑 만날 사람, 내가 이 사람을 만나야 될지, 아니면 만나지 말아야 될지, 그걸 판단하는 데 필요한 게 정보잖아요. 나랑 가치관이 비슷하면 계속 만나는 게 편하고, 나와 가치관이 다르면 계속 상대를 설득해야 되고, 그런 측면에서 정보, 정보는 사람들을 묵는데 접촉대 같은 역할을 하는 것이고, 그걸 넥서스라고 부른다. 이렇게 보시면 될 것 같아요. 물론 당연히 저자의 의도와는 다를 수 있는데, 저는 그렇게 읽게 되었습니다. 그리고 챕터 두 번째 것도 읽게 되었는데요, 다시 한번 같이 볼게요. 챕터 2, 그래서 여러 가지 정보들이 있는데, 이야기, 드라마나 이런 것처럼, 이야기가 그런 수많은 사람들을 무한하게 연결시킨다, 이런 챕터이고요. 여기도 정말 좋은 문장들이 많았는데, 딱 세 개만 제가 편집해서 뽑아봤습니다. 얼마나 무한하냐면, 가톨릭 교회는 44억 명 정도가 어떤 하나의 스토리를 공유해서 연결이 됐고, 그리고 세계무용망은 80억 모든 사피엔스를 연결한다. 그래서 세계무용망을 연결한 장치는 화폐, 기업, 상표에 관련된 이야기들이다. 저 80억에 저도 들어가 있습니다. 저도 이제 자본주의에 살고 있잖아요. 그래서 당연히 화폐, 기업, 상표에 관련된 이야기를 믿고 있는데, 여러분들도 화폐, 기업, 상표에 관련된 이야기들을 믿고 계신지 한번 생각해보시면 좋을 것 같아요. 그리고 사람들을 결속시키는 허구적인 정보는 진실에 비해 두 가지 고유한 이점을 가지고 있다. 허구는 간단하고 진실은 복잡할 때가 많다. 진실은 고통스럽고 불편한데, 허구는 매력적이다. 이것도 많은 생각을 하게 되는 문장인 것 같은데요. 예를 들면 다이어트를 한다고 했을 때, 다이어트를 하는 방법은 정말 고통스럽죠. 먹고 싶은 거 참아야 되고, 하기 싫은 운동을 해야 되고, 매우 힘들죠. 그런데 사람들이 많이 열광하는 다이어트 방식은 힘 안 들이고 다이어트하는 방법, 이런 것들이죠. 많은 사람들이 거기에 많이 끌리고 돈을 쓰는데, 장기적으로 더 고통이죠. 그리고 종교도 마찬가지죠. 종교도 진실을 내가 다 이해하고 개개인적으로 선택하려면 힘든데, 종교에서는 종교의 가르침을 잘 따르면 좋은 결과가 나온다고 확정을 해주니까 참 편하죠. 그래서 진실이 고통스러울 때가 많아요. 그래서 꼭 진실이 좋지는 않다는 생각이 들고, 진실이 잘 퍼지지도 않고, 그런 것 같습니다. 그리고 사람들이 진실에 불안할 정도로 다가가려고 하면, 사회는 진실을 제한하여 질서를 유지할 것이다. 많은 정부와 교회가 지하교육을 금지하여 질서를 위해 진실을 희생하는 쪽을 선택하는 이유다. 이것들도 그런 경우가 많이 있죠. 미성년자들이 보지 말아야 하는 영화들 이런 것도 있잖아요. 그리고 민감한 주제들이긴 한데, 질서 유지를 위해서 그냥 거짓말인 줄 알면서도 넘어가는 그런 진실들도 있죠. 그런 게 뭐가 있는지에 대해서 이야기를 좀 나눠보면 좋을 것 같아요. 그래서 이런 인사이트를 주는 문장들이 많이 있습니다. 그래서 세 가지 질문 여러분들 한번 적어보시면 좋을 것 같습니다. 그래서 저도 여기 노션 페이지에 적고 있어요. 그 이전에 프롤로그 했던 질문이 있고, 이거는 유튜브 밑에 링크를 알아드릴 텐데, 좋았던 문장 적고 질문 세 가지 적으면서 답변을 적어봤으니까요. 관심 있는 분들은 한번 읽어주시면 좋을 것 같고요. 그래서 다 제 생각을 적은 거기 때문에 여러분들 생각이 당연히 다를 수 있을 것 같아요. 중요한 건 이런 질문들을 하고 고민해보는 이런 습관들은 중요한 것 같습니다. 그리고 이제 주말마다 책을 읽기 때문에 다음 주 주말에 이 후의 이야기들을 한번 읽어보고 정리를 해보겠습니다. 그래서 지금 유튜브 두 번째인데요. 확실히 첫 번째 보다는 조금 더 말을 잘하는 것 같아요. 점점 발전하겠죠? 지금 넥서스 책을 읽는데 저는 너무 지금 내용이 좋은 것 같아요. 아무래도 제가 이전 작들을 다 읽어서 그런지 생각할 것들이 많고, 예전 기억들도 다시 회상이 되고 하는 것 같습니다. 그래서 앞으로 주말에 아침에 책을 읽고, 요약을 하고 영상으로 정리하는 이런 루틴을 한동안 유지하도록 하겠습니다. 그러면은 여기서 오늘 영상 마치겠습니다. 감사합니다. 종료\n'},
    config={
        "configurable": {
            "thread_id": "1",
        },
    }),



In [ ]:
response = {
    "user_feedback": "약간 3d 느낌을 추가해주고, 좌우 대칭이 잘 맞도록 이미지를 만들어줘",
    "chosen_prompt": 3,
}

graph.invoke(
    Command(resume=response),
    config=config,
)